# Requirements

- install node2vec(*) code and add executable to your $PATH (code: https://snap.stanford.edu/node2vec)
- compile GED code (graph embedding divergence), 
  the base implementation of the framework in C (the code is included, and can also be found at      https://github.com/ftheberge/Comparing_Graph_Embeddings) 
- new package to install: 'pip install --no-dependencies graphrole'
- adjust location of data and code in next cell

(*) With the node2vec code, we got some unstable results on some older Linux platforms. In such cases, one possible way to improve the results is to reduce the path length for the random walks (default = 80) to something smaller like 10 or 20. The option is "-l:10" or "-l:20" respectively. 
  

In [ ]:
## the data directory
datadir='../Datasets/'

## location of the GED code
GED='../GED/GED'

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from collections import Counter
import os
import umap
import pickle
import partition_igraph
import subprocess
import scipy.sparse.linalg as lg
from sklearn.cluster import KMeans, DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_mutual_info_score as AMI
from graphrole import RecursiveFeatureExtractor, RoleExtractor
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import calinski_harabasz_score as CHS
from sklearn.metrics import silhouette_score as SIL
#%config Completer.use_jedi = False

## node and edge greyscale colors
cls_edges = 'gainsboro'
cls = ['silver','dimgray','black']

# A few useful functions

In [ ]:
## as defined in the node2vec paper
def binary_operator(u, v, op='had'):
    if op=='had':
        return u * v
    if op=='l1':
        return np.abs(u - v)
    if op=='l2':
        return (u - v) ** 2
    if op=='avg':
        return (u + v) / 2.0
    
def readEmbedding(fn="_embed", N2K=None):
    D = pd.read_csv(fn, sep=' ', skiprows=1, header=None)
    D = D.dropna(axis=1)
    if N2K!=None:
        x = [N2K[i] for i in D[0]]
        D[0] = x    
        D = D.sort_values(by=0)
    Y = np.array(D.iloc[:,1:])
    return Y

## Read embedding from file in node2vec format
## Map to layout format
## for visualization, we use UMAP if dim > 2
def embed2layout(fn="_embed"):
    D = pd.read_csv(fn, sep=' ', skiprows=1, header=None)
    D = D.dropna(axis=1)
    D = D.sort_values(by=0)
    Y = np.array(D.iloc[:,1:])
    if Y.shape[1]>2:
        Y = umap.UMAP().fit_transform(Y)
    ly = []
    for v in range(Y.shape[0]):
        ly.append((Y[v][0],Y[v][1]))
    return ly


## Computing JS divergence with GED code given edgelist, communities and embedding
def JS(edge_file, comm_file, embed_file, entropy=False):
    if entropy:
        x = GED+' -E -g '+edge_file+' -c '+comm_file+' -e '+embed_file
    else:
        x = GED+' -g '+edge_file+' -c '+comm_file+' -e '+embed_file
    s = subprocess.run(x, shell=True, stdout=subprocess.PIPE)
    x = s.stdout.decode().split(' ')
    div = float(x[1])
    return(div)


## Hope embedding with various similarity functions
def Hope(g, sim='katz', dim=2, verbose=False, beta=.01, alpha=.5):
    ## For undirected graphs, embedding as source and target are identical
    if g.is_directed() == False:
        dim = dim*2
    A = np.array(g.get_adjacency().data)
    beta = beta
    alpha = alpha
    n = g.vcount()
    ## Katz
    if sim == 'katz':
        M_g = np.eye(n) - beta * A
        M_l = beta * A
    ## Adamic-Adar
    if sim == 'aa':
        M_g = np.eye(n)
        ## fix bug 1/x and take log();
        D = np.diag([1/np.log(x) if x>1 else 0 for x in g.degree()]) 
        # D = np.diag([1/np.log(max(2,x)) for x in g.degree()]) 
        M_l = np.dot(np.dot(A,D),A)
        np.fill_diagonal(M_l,0)
    ## Common neighbors
    if sim == 'cn':
        M_g = np.eye(n)
        M_l = np.dot(A,A)
    ## presonalized page rank
    if sim == 'ppr':
        P = []
        for i in range(n):
            s = np.sum(A[i])
            if s>0:
                P.append([x/s for x in A[i]])
            else:
                P.append([1/n for x in A[i]])
        P = np.transpose(np.array(P)) ## fix bug - take transpose
        M_g = np.eye(n)-alpha*P
        M_l = (1-alpha)*np.eye(n)
    S = np.dot(np.linalg.inv(M_g), M_l)
    u, s, vt = lg.svds(S, k=dim // 2)
    X1 = np.dot(u, np.diag(np.sqrt(s)))
    X2 = np.dot(vt.T, np.diag(np.sqrt(s)))
    X = np.concatenate((X1, X2), axis=1)
    p_d_p_t = np.dot(u, np.dot(np.diag(s), vt))
    eig_err = np.linalg.norm(p_d_p_t - S)
    if verbose:
        print('SVD error (low rank): %f' % eig_err)
    ## undirected graphs have identical source and target embeddings
    if g.is_directed() == False:
        d = dim//2
        return X[:,:d]
    else:
        return X

## save to disk to compute divergence
def saveEmbedding(X, g, fn='_embed'):
    with open(fn,'w') as f:
        f.write(str(X.shape[0]) + " " + str(X.shape[1])+'\n')
        for i in range(X.shape[0]):
            f.write(g.vs[i]['name']+' ')
            for j in range(X.shape[1]):
                f.write(str(X[i][j])+' ')
            f.write('\n')

## Laplacian eigenmaps embedding
def LE(g, dim=2):
    L_sym = np.array(g.laplacian(normalized=True))
    w, v = lg.eigs(L_sym, k=dim + 1, which='SM')
    idx = np.argsort(w) # sort eigenvalues
    w = w[idx]
    v = v[:, idx]
    X = v[:, 1:]
    return X.real

def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)


# Figure 6.1

In [ ]:
## To illustrate random walks
g = ig.Graph.Erdos_Renyi(n=4,p=0,directed=True)
g.vs['label'] = ['A','B','C','D']
g.vs['color'] = 'white'
g.add_edges([(0,1),(1,2),(1,3),(2,1),(3,2)])
#ig.plot(g,'tiny.eps',bbox=(0,0,300,200),vertex_label_size=10)
ig.plot(g,bbox=(0,0,300,200),vertex_label_size=10)

# Load and prepare datasets

* g: small ABCD graph (100 nodes), mainly for visualization and quick exampes
* G: larger ABCD graph (1000 nodes), for experiments
* z: zachary graph, for visualzation

## 1. Small ABCD graph 

In [ ]:
## read graph and communities
g = ig.Graph.Read_Ncol(datadir+'ABCD/abcd_100.dat',directed=False)
c = np.loadtxt(datadir+'ABCD/abcd_100_comms.dat',dtype='uint16',usecols=(1))
g.vs['comm'] = [c[int(x['name'])-1] for x in g.vs]
## print a few stats
print(g.vcount(),'vertices,',g.ecount(),'edges,','avg degreee',np.mean(g.degree()),'communities',max(g.vs['comm']))
## ground truth
gt = {k:(v-1) for k,v in enumerate(g.vs['comm'])}
## map between int(name) to key
n2k = {int(v):k for k,v in enumerate(g.vs['name'])}

## define the colors and node sizes here
g.vs['size'] = 7
g.es['color'] = cls_edges
g.vs['color'] = [cls[i-1] for i in g.vs['comm']]
#ig.plot(g, 'abcd.eps', bbox=(0,0,300,200))
ig.plot(g, bbox=(0,0,300,200))

## 2. Larger, noisy ABCD graph

This is a larger graph with lots of noise edges (xi=0.6)

We'll use a version with stronger communities (xi=0.2) for link prediction.


In [ ]:
## read graph and communities
G = ig.Graph.Read_Ncol(datadir+'ABCD/abcd_1000.dat',directed=False)
c = np.loadtxt(datadir+'ABCD/abcd_1000_comms.dat',dtype='uint16',usecols=(1))
G.vs['comm'] = [c[int(x['name'])-1] for x in G.vs]
## print a few stats
print(G.vcount(),'vertices,',G.ecount(),'edges,','avg degreee',np.mean(G.degree()),'communities',max(G.vs['comm']))
## ground truth
GT = {k:(v-1) for k,v in enumerate(G.vs['comm'])}
## map between int(name) to key
N2K = {int(v):k for k,v in enumerate(G.vs['name'])}
## define the colors and node sizes here
cls_edges = 'gainsboro'
G.vs['size'] = 5
G.es['color'] = cls_edges
G.vs['color'] = 'black'
ig.plot(G, bbox=(0,0,400,300)) ## communities are far from obvious in 2d layout!

## 3. Zachary (karate) graph


In [ ]:
z = ig.Graph.Famous('zachary')
z.vs['size'] = 7
z.vs['name'] = [str(i) for i in range(z.vcount())]
z.es['color'] = cls_edges
z.vs['comm'] = [0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]
z.vs['color'] = [cls[i*2] for i in z.vs['comm']]
#ig.plot(z, 'zachary.eps', bbox=(0,0,300,200))
ig.plot(z, bbox=(0,0,300,200))

# Show various 2d layouts using small Zachary graph

In [ ]:
ly = z.layout('kk')
#ig.plot(z, 'layout_kk.eps', layout=ly, bbox=(0,0,300,200))
ig.plot(z, layout=ly, bbox=(0,0,300,200))


In [ ]:
ly = z.layout('fr')
#ig.plot(z, 'layout_fr.eps', layout=ly, bbox=(0,0,300,200))
ig.plot(z, layout=ly, bbox=(0,0,300,200))

In [ ]:
ly = z.layout('mds')
#ig.plot(z, 'layout_mds.eps', layout=ly, bbox=(0,0,300,200))
ig.plot(z, layout=ly, bbox=(0,0,300,200))

In [ ]:
ly = z.layout('circle')
#ig.plot(z, 'layout_circle.eps', layout=ly, bbox=(0,0,300,200))
ig.plot(z, layout=ly, bbox=(0,0,300,200))

In [ ]:
ly = z.layout('grid')
#ig.plot(z, 'layout_grid.eps', layout=ly, bbox=(0,0,300,200))
ig.plot(z, layout=ly, bbox=(0,0,300,200))

In [ ]:
ly = z.layout('sugiyama')
#ig.plot(z, 'layout_tree.eps', layout=ly, bbox=(0,0,300,200))
ig.plot(z, layout=ly, bbox=(0,0,300,200))

# Perform several embeddings -- Zachary graph
* node2vec from source code
* HOPE with different similarities
* Laplacian Eigenmaps
* visualize some good and bad results

We use the framework to compute the "graph embedding divergence" (GED.c)

In [ ]:
L = []
DIM = [5,10,15]
best_jsd = 1
worst_jsd = 0

## Hope
for dim in DIM:
    for sim in ['katz','ppr','cn','aa']:
        X = Hope(z,sim=sim,dim=dim) 
        saveEmbedding(X,z)
        jsd = JS(datadir+'Zachary/zachary.edgelist',datadir+'Zachary/zachary.ecg','_embed')        
        ## keep track of best and worst
        if jsd < best_jsd:
            os.system('cp _embed _embed_best')
            best_jsd = jsd
        if jsd > worst_jsd:
            os.system('cp _embed _embed_worst')
            worst_jsd = jsd
        L.append([dim,'hope',sim,jsd])

## LE
for dim in DIM:
    X = LE(z,dim=dim)
    saveEmbedding(X,z)
    jsd = JS(datadir+'Zachary/zachary.edgelist',datadir+'Zachary/zachary.ecg','_embed')
    ## keep track of best and worst
    if jsd < best_jsd:
        os.system('cp _embed _embed_best')
        best_jsd = jsd
    if jsd > worst_jsd:
        os.system('cp _embed _embed_worst')
        worst_jsd = jsd
    L.append([dim,'le',' ',jsd])
    
## node2vec is in my path
for dim in DIM:
    for (p,q) in [(1,0.1),(0.1,1),(1,1)]:
        x = 'node2vec -i:'+datadir+'Zachary/zachary.edgelist -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
        ## if you get unstable results, you can try with shorter random walks, for example:
        ## x = 'node2vec -l:3 -i:'+datadir+'Zachary/zachary.edgelist -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
        r = os.system(x)
        jsd = JS(datadir+'Zachary/zachary.edgelist',datadir+'Zachary/zachary.ecg','_embed')
        ## keep track of best and worst
        if jsd < best_jsd:
            os.system('cp _embed _embed_best')
            best_jsd = jsd
        if jsd > worst_jsd:
            os.system('cp _embed _embed_worst')
            worst_jsd = jsd

        L.append([dim,'n2v',str(p)+' '+str(q),jsd])


In [ ]:
D = pd.DataFrame(L,columns=['dim','algo','param','jsd'])
D = D.sort_values(by='jsd',axis=0)
D.head()

In [ ]:
os.system('cp _embed_best _embed')
l = embed2layout()
z.vs['ly'] = [l[int(v['name'])] for v in z.vs]
#ig.plot(z, 'zac_high.eps', layout=z.vs['ly'], bbox=(0,0,300,200))
ig.plot(z,layout=z.vs['ly'], bbox=(0,0,300,200))

In [ ]:
D.tail()

In [ ]:
os.system('cp _embed_worst _embed')
l = embed2layout()
z.vs['ly'] = [l[int(v['name'])] for v in z.vs]
#ig.plot(z, 'zac_high.eps', layout=z.vs['ly'], bbox=(0,0,300,200))
ig.plot(z,layout=z.vs['ly'], bbox=(0,0,300,200))

# Perform several embeddings -- small ABCD  graph
* node2vec from source code
* HOPE different similarities
* Laplacian Eigenmaps
* visualize some good and bad results

In [ ]:
L = []
DIM = [2,4,8,16,24,32]
best_jsd = 1
worst_jsd = 0

## Hope
for dim in DIM:
    for sim in ['katz','aa','cn','ppr']:
        X = Hope(g,sim=sim,dim=dim) 
        saveEmbedding(X,g)
        jsd = JS(datadir+'ABCD/abcd_100.dat',datadir+'ABCD/abcd_100.ecg','_embed')
        ## keep track of best and worst
        if jsd < best_jsd:
            os.system('cp _embed _embed_best')
            best_jsd = jsd
        if jsd > worst_jsd:
            os.system('cp _embed _embed_worst')
            worst_jsd = jsd
        L.append([dim,'hope',sim,jsd])

## LE
for dim in DIM:
    X = LE(g,dim=dim)
    saveEmbedding(X,g)
    jsd = JS(datadir+'ABCD/abcd_100.dat',datadir+'ABCD/abcd_100.ecg','_embed')
    ## keep track of best and worst
    if jsd < best_jsd:
        os.system('cp _embed _embed_best')
        best_jsd = jsd
    if jsd > worst_jsd:
        os.system('cp _embed _embed_worst')
        worst_jsd = jsd
    L.append([dim,'le',' ',jsd])
    
## node2vec is in my path
for dim in DIM:
    for (p,q) in [(1,0.1),(1,.5),(0.1,1),(.5,1),(1,1)]:
        x = 'node2vec -i:'+datadir+'ABCD/abcd_100.dat -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
        ## if you get unstable results, you can try with shorter random walks, for example:
        ## x = 'node2vec -l:10 -i:'+datadir+'ABCD/abcd_100.dat -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
        r = os.system(x)
        jsd = JS(datadir+'ABCD/abcd_100.dat',datadir+'ABCD/abcd_100.ecg','_embed')
        ## keep track of best and worst
        if jsd < best_jsd:
            os.system('cp _embed _embed_best')
            best_jsd = jsd
        if jsd > worst_jsd:
            os.system('cp _embed _embed_worst')
            worst_jsd = jsd
        L.append([dim,'n2v',str(p)+' '+str(q),jsd])


In [ ]:
D = pd.DataFrame(L,columns=['dim','algo','param','jsd'])
D = D.sort_values(by='jsd',axis=0)
D.head()

In [ ]:
os.system('cp _embed_best _embed')
l = embed2layout()
g.vs['ly'] = [l[int(v['name'])-1] for v in g.vs]
ig.plot(g, layout=g.vs['ly'], bbox=(0,0,300,200))


In [ ]:
D.tail()

In [ ]:
os.system('cp _embed_worst _embed')
l = embed2layout()
g.vs['ly'] = [l[int(v['name'])-1] for v in g.vs]
ig.plot(g, layout=g.vs['ly'], bbox=(0,0,300,200))


# Large ABCD graph -- find a good embedding with the framework
* we ran the code below and saved the best embedding in datdadir+"ABCD/abcd_1000_embed_best" for graph with xi=0.6
* this can be re-run by uncommenting the cell below
* we'll consider more embeddings in the large classification experiment later

# Classification on larger ABCD graph

* we use a good saved embedding 
* we use a random forest model on embedded space
* we split the data as train and test
* the goal is to recover the communities for each node


In [ ]:
## used the saved "best" embedding from above
X = readEmbedding(fn=datadir+"ABCD/abcd_1000_embed_best")
y = G.vs['comm']
## train/test split
np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=0)


In [ ]:
# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)

# Class predictions on test data
y_pred = model.predict(X_test)

In [ ]:
## Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
## percent correct -- this can vary slightly as we split train/test randomly
print('\naccuracy:',sum(cm.diagonal())/sum(sum(cm)),'\n')
#print(bmatrix(cm))

In [ ]:
## compare with random classifier -- assuming we know only the number of classes (12)
y_pred = [x+1 for x in np.random.choice(12,size=len(y_test),replace=True)]
cm = confusion_matrix(y_test, y_pred)
# print(cm)
## percent correct
print('\nAccuracy:',sum(cm.diagonal())/sum(sum(cm)))

In [ ]:
## compare with random classifier -- using class proportions in training data
ctr = Counter(y_train)
x = [ctr[i+1] for i in range(12)]
s = np.sum(x)
p = [i/s for i in x]
y_pred = [x+1 for x in np.random.choice(12,size=len(y_test),replace=True,p=p)]
cm = confusion_matrix(y_test, y_pred)
# print(cm)
## percent correct
print('\nAccuracy:',sum(cm.diagonal())/sum(sum(cm)))

# Clustering
* we run graph clustering (Louvain, ECG)
* we compare with vector space embedding using same embedding
* we use k-means (various k) and DBSCAN
* recall there are 12 ground truth communities

In [ ]:
## again we use 'good' embedding from before
X = readEmbedding(fn=datadir+"ABCD/abcd_1000_embed_best")

In [ ]:
L = []
K = [6,9,12,15,24] ## for k-means (real number of clusters is 12)
REP = 30

for i in range(REP):
    
    ## kmeans
    for k in K:
        cl = KMeans(n_clusters=k).fit(X)
        d = {k:v for k,v in enumerate(cl.labels_)}
        scr = CHS(X,cl.labels_)
        ami = AMI(list(GT.values()),list(d.values()))
        L.append(['km'+str(k),scr,ami])

    ## ECG
    ec = G.community_ecg().membership
    scr = G.modularity(ec)
    ami = AMI(list(GT.values()),ec)
    L.append(['ecg',scr,ami])
    
    ## Louvain -- permute as this is not done in igraph
    p = np.random.permutation(G.vcount()).tolist()
    GG = G.permute_vertices(p)
    l = GG.community_multilevel().membership
    ll = [-1]*len(l)
    for i in range(len(l)):
        ll[i] = l[p[i]]
    scr = G.modularity(ll)
    ami = AMI(list(GT.values()),ll)
    L.append(['ml',scr,ami])

In [ ]:
## results with best score for 3 algorithms
D = pd.DataFrame(L,columns=['algo','scr','ami'])

x = list(D[[x.startswith('km') for x in D['algo']]].sort_values(by='scr',ascending=False)['ami'])[0]
print('K-Means:',x)

x = list(D[D['algo']=='ml'].sort_values(by='scr',ascending=False)['ami'])[0]
print('Louvain:',x)

x = list(D[D['algo']=='ecg'].sort_values(by='scr',ascending=False)['ami'])[0]
print('ECG:',x)


In [ ]:
## boxplot AMI results
A = []
algo = ['km6','km9','km12','km15','km24','ml','ecg']
for a in algo:
    A.append(D[D['algo']==a]['ami'])

B = pd.DataFrame(np.transpose(A), 
                 columns=['k-means(6)','k-means(9)','k-means(12)','k-means(15)',
                          'k-means(24)','Louvain','ECG'])
B.boxplot(rot=30,figsize=(7,5))
plt.ylabel('Adjusted Mutual Information (AMI)');
#plt.savefig('embed_cluster.eps')

In [ ]:
## DBSCAN -- we tried a few min_sample and dim, good results with 8 and 16 resp.
## try various epsilon and test via calinski_harabasz_score (CHS) or silhouette_score 
top = 0
for dim in [16]:
    for ms in [8]:
        U = umap.UMAP(n_components=dim).fit_transform(X)
        for e in np.arange(.4,.5,.0025):
            cl = DBSCAN(eps=e, min_samples=ms ).fit(U)
            labels = cl.labels_
            s = CHS(U,labels) ## score
            if s > top:
                top=s
                e_top=e
                d_top=dim
                m_top=ms
U = umap.UMAP(n_components=d_top).fit_transform(X)
cl = DBSCAN(eps=e_top, min_samples=m_top).fit(U)

b = [x>-1 for x in cl.labels_]
l = list(GT.values())
v = [l[i] for i in range(len(l)) if b[i]]
print('AMI without outliers:',AMI(v,cl.labels_[b]))
print('AMI with outliers:',AMI(list(GT.values()),cl.labels_))


# Link prediction

* we drop 10% edges and re-compute the embedding
* we train a logistic regression model
* we apply final model to test set


## First try with noisy graph

Recall that xi=0.6, the proportion of noise edges


In [ ]:
## pick 10% edges at random, save new graph as Gp
test_size = int(np.round(.1*G.ecount()))
np.random.seed(123456)
test_eid = np.random.choice(G.ecount(),size=test_size,replace=False)
Gp = G.copy()
Gp.delete_edges(test_eid)

## compute embedding on Gp with parameters that yielded a good embedding for G
X = Hope(Gp,sim='ppr', dim=48)


In [ ]:
## Train model with Hadamard binary operator (other choices are 'l1', 'l2 and 'avg')
op = 'had'

## Build training data, first the edges
F = []
for e in Gp.es:
    F.append(binary_operator(X[e.tuple[0]],X[e.tuple[1]],op=op))
size = len(F)
f = [1]*size

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e = [tuple(np.random.choice(Gp.vcount(),size=2,replace=False)) for i in range(2*size)]
e = [(min(x),max(x)) for x in e if Gp.get_eid(x[0],x[1],directed=False,error=False) == -1]
non_edges = list(set(e))[:size]
for e in non_edges:
    F.append(binary_operator(X[e[0]],X[e[1]],op=op))
F = np.array(F)
f.extend([0]*size)

## train model
logreg = LogisticRegression()
logreg.fit(F,f)

## prepare test set, first with all dropped edges from G 
X_test = []
for i in test_eid:
    e = G.es[i]
    X_test.append(binary_operator(X[e.tuple[0]],X[e.tuple[1]],op=op))
size = len(X_test)
y_test = [1]*size

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e = [tuple(np.random.choice(G.vcount(),size=2,replace=False)) for i in range(2*size)]
e = [(min(x),max(x)) for x in e if G.get_eid(x[0],x[1],directed=False,error=False) == -1]
non_edges = list(set(e))[:size]
for e in non_edges:
    X_test.append(binary_operator(X[e[0]],X[e[1]],op=op))
X_test = np.array(X_test)
y_test.extend([0]*size)

## apply the model to test data
print('Accuracy of logistic regression classifier with',op,'on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
print('AUC:',roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1]))

## Link prediction with less noisy graph

In [ ]:
## read graph and communities - graph with xi=0.2
G2 = ig.Graph.Read_Ncol(datadir+'ABCD/abcd_1000_xi2.dat',directed=False)
c = np.loadtxt(datadir+'ABCD/abcd_1000_xi2_comms.dat',dtype='uint16',usecols=(1))
G2.vs['comm'] = [c[int(x['name'])-1] for x in G2.vs]

## pick 10% edges at random, save new graph as Gp
test_size = int(np.round(.1*G2.ecount()))
np.random.seed(123456) ## for reproducibility
test_eid = np.random.choice(G2.ecount(),size=test_size,replace=False)
Gp = G2.copy()
Gp.delete_edges(test_eid)

## compute embedding on Gp with parameters that yielded a good embedding for G2
X = Hope(Gp,sim='ppr', dim=48)


In [ ]:
## Train model with Hadamard binary operator (other choices are 'l1', 'l2 and 'avg')
op = 'had'

## Build training data, first the edges
F = []
for e in Gp.es:
    F.append(binary_operator(X[e.tuple[0]],X[e.tuple[1]],op=op))
size = len(F)
f = [1]*size

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e = [tuple(np.random.choice(Gp.vcount(),size=2,replace=False)) for i in range(2*size)]
e = [(min(x),max(x)) for x in e if Gp.get_eid(x[0],x[1],directed=False,error=False) == -1]
non_edges = list(set(e))[:size]
for e in non_edges:
    F.append(binary_operator(X[e[0]],X[e[1]],op=op))
F = np.array(F)
f.extend([0]*size)

## train model
logreg = LogisticRegression()
logreg.fit(F,f)

## prepare test set, first with all dropped edges from G 
X_test = []
for i in test_eid:
    e = G2.es[i]
    X_test.append(binary_operator(X[e.tuple[0]],X[e.tuple[1]],op=op))
size = len(X_test)
y_test = [1]*size

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e = [tuple(np.random.choice(G2.vcount(),size=2,replace=False)) for i in range(2*size)]
e = [(min(x),max(x)) for x in e if G2.get_eid(x[0],x[1],directed=False,error=False) == -1]
non_edges = list(set(e))[:size]
for e in non_edges:
    X_test.append(binary_operator(X[e[0]],X[e[1]],op=op))
X_test = np.array(X_test)
y_test.extend([0]*size)

## apply the model to test data
print('Accuracy of logistic regression classifier with',op,'on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
print('AUC:',roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1]))

In [ ]:
logit_roc_auc = roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1])
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, color='gray',label='Logistic Regression (AUC = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend(loc="lower right")
#plt.savefig('embed_link.eps')
plt.show();

## Larger study -- use classification accuracy for picking embedding

- we use training-validation-test split
- this can be long to run -- a pickle file with the results is included in data directory
- to re-run from scratch, uncomment the next cell; results can differ in that case due to non-deterministic algorithms like node2vec

In [ ]:
## load L and train/val/test ids
with open(datadir+"ABCD/abcd_1000_embeddings.pkl","rb") as f:
    id_train,id_val,id_trainval,id_test,L = pickle.load(f)
y_all = G.vs['comm']
y_train = [y_all[i] for i in id_train]
y_trainval = [y_all[i] for i in id_trainval]
y_val = [y_all[i] for i in id_val]
y_test = [y_all[i] for i in id_test]

In [ ]:
R = pd.DataFrame(L,columns=['dim','algo','param','div','acc'])
from scipy.stats import kendalltau as tau
print(tau(R['div'],R['acc']))


In [ ]:
## sort by Divergence on validation set
R = R.sort_values(by='div',axis=0,ascending=True)
size = R.shape[0]
R['rank_div'] = np.arange(1,size+1,1)
R.head()


In [ ]:
## sort by Accuracy on validation set
R = R.sort_values(by='acc',axis=0,ascending=False)
size = R.shape[0]
R['rank_acc'] = np.arange(1,size+1,1)
R.head()


In [ ]:
## quite a range of accuracy on the validation set!
R.tail()

##  Apply to test set. 

This takes several minutes to run so a pickle file is provided.

Uncomment the cell below to re-run; results can differ in that case due to non-deterministic algorithms like node2vec

In [ ]:
## load test results
with open(datadir+"ABCD/abcd_1000_embeddings_test.pkl","rb") as f:
    top_acc = pickle.load(f)
R['test'] = top_acc
print('mean accuracy over all models on test set:',np.mean(R['test']))


In [ ]:
R = R.sort_values(by='test',axis=0,ascending=False)
R['rank_test'] = np.arange(1,size+1,1)
R.head()

In [ ]:
## top results on test set w.r.t. divergence on validation set
R = R.sort_values(by='div',axis=0,ascending=True)
top_div = R['test'][:10]

## top results on test set w.r.t. accuracy on validation set
R = R.sort_values(by='acc',axis=0,ascending=False)
top_acc = R['test'][:10]


In [ ]:
## pd with mu
B = pd.DataFrame(np.transpose(np.array([top_acc,top_div])), 
                 columns=['Top-10 validation set accuracy','Top-10 divergence score'])
B.boxplot(rot=0,figsize=(7,5), widths=.33)
plt.ylabel('Test set accuracy',fontsize=14);
#plt.savefig('embed_classify.eps')

In [ ]:
plt.plot(R['rank_acc'],R['test'],'.',color='black')
plt.xlabel('Rank',fontsize=14)
plt.ylabel('Test set accuracy',fontsize=14);
#plt.savefig('rank_accuracy.eps');

In [ ]:
plt.plot(R['rank_div'],R['test'],'.',color='black')
plt.xlabel('Rank',fontsize=14)
plt.ylabel('Test set accuracy',fontsize=14);
#plt.savefig('rank_divergence.eps');

In [ ]:
## negative correlations
print(np.corrcoef(R['rank_acc'],R['test'])[0,1],
      np.corrcoef(R['rank_div'],R['test'])[0,1])

In [ ]:
## random classification -- AMI
ctr = Counter(y_trainval)
x = [ctr[i+1] for i in range(12)]
s = np.sum(x)
p = [i/s for i in x]
y_pred = [x+1 for x in np.random.choice(12,size=len(y_test),replace=True,p=p)]
cm = confusion_matrix(y_test, y_pred)
print('\nRandom classifier accuracy on test set:',sum(cm.diagonal())/sum(sum(cm)))


## ReFex: illustrate roles on Zachary graph

We use the 'graphrole' package


In [ ]:
# extract features
feature_extractor = RecursiveFeatureExtractor(z, max_generations=4)
features = feature_extractor.extract_features()
print(f'\nFeatures extracted from {feature_extractor.generation_count} recursive generations:')
features.head(10)

In [ ]:
# assign node roles in a dictionary
role_extractor = RoleExtractor(n_roles=3)
role_extractor.extract_role_factors(features)
node_roles = role_extractor.roles
role_extractor.role_percentage.head()

In [ ]:
#import seaborn as sns
unique_roles = sorted(set(node_roles.values()))
# uncomment for color plot
# cls = ['red','blue','green']
# map roles to colors
role_colors = {role: cls[i] for i, role in enumerate(unique_roles)}

# store colors for all nodes in G
z.vs()['color'] = [role_colors[node_roles[node]] for node in range(z.vcount())]

## Plot with node labels
z.vs()['size'] = 10
#z.vs()['label'] = [v.index for v in z.vs()]
z.vs()['label_size'] = 0
#ig.plot(z, 'refex.eps', bbox=(0,0,300,300)) 
ig.plot(z, bbox=(0,0,300,300)) 



# Anomaly detection

## Dataset -- American College Football Graph

[REF]: "Community structure in social and biological networks", M. Girvan and M. E. J. Newman
PNAS June 11, 2002 99 (12) 7821-7826; https://doi.org/10.1073/pnas.122653799


Teams are part of 12 conferences (the 'communities'):
*   0 = Atlantic Coast
*   1 = Big East
*   2 = Big Ten
*   3 = Big Twelve
*   4 = Conference USA
*   5 = Independents
*   6 = Mid-American
*   7 = Mountain West
*   8 = Pacific Ten
*   9 = Southeastern
*  10 = Sun Belt
*  11 = Western Athletic

14 teams out of 115 appear as anomalies as can be seen in Figure 5 of [REF], namely:
- 5 teams in #5 conference (Independent) play teams in other conferences (green triangles)
- 7 teams in #10 conference (Sun Belt) are broken in 2 clumps (pink triangles) 
- 2 teams from #11 conference play mainly with #10 conference (red triangles)

Here, we try to recover those anomalous teams by running several embeddings (we use node2vec):

- for each embedding:
 - compute divergence using our framework
 - also compute entropy of b-vector for each node (probability distribution of edges w.r.t. every community in the geometric Chung-Lu model)
- plot entropy vs divergence
- for some good/bad embedding, boxplot entropy of anomalous vs other nodes



In [ ]:
## read graph and communities
g = ig.Graph.Read_Ncol(datadir+'Football/football.edgelist',directed=False)
c = np.loadtxt(datadir+'Football/football.community',dtype='uint16',usecols=(0))
g.vs['community'] = [c[int(x['name'])] for x in g.vs]

## Read and plot the College Football Graph
g.vs['shape'] = 'circle'
g.vs['anomaly'] = 0
pal = ig.RainbowPalette(n=max(g.vs['community'])+1) 
g.vs['color'] = [pal.get(int(i)) for i in g.vs['community']]
for v in g.vs:
    if v['community'] in [5,10] or v['name'] in ['28','58']:
        v['shape']='triangle'
        v['anomaly']=1
ly = g.layout_fruchterman_reingold()
ig.plot(g, layout=ly, bbox=(0,0,500,300), vertex_size=8, edge_color='lightgray')
#ig.plot(g, target="anomaly_0.eps", layout=ly, bbox=(0,0,500,300), vertex_size=8, edge_color='lightgray')


In [ ]:
## greyscale
pal = ig.GradientPalette("white","black",max(g.vs['community'])+1)
g.vs['color'] = [pal.get(int(i)) for i in g.vs['community']]
ig.plot(g, layout=ly, bbox=(0,0,500,300), vertex_size=8, edge_color='lightgray')
#ig.plot(g, target="anomaly_1.eps", layout=ly, bbox=(0,0,500,300), vertex_size=8, edge_color='lightgray')


In [ ]:
best_jsd = 1
worst_jsd = 0
## node2vec with varying parameters (60 embeddings)
L = []
for dim in np.arange(2,25,2):
    for (p,q) in [(1,0.5),(0.5,1),(1,0.01),(0.01,1),(1,1)]:
        x = 'node2vec -i:'+datadir+'Football/football.edgelist -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
        ## if you get unstable results, you can try with shorter random walks, for example:
        ## x = 'node2vec -l:15 -i:'+datadir+'Football/football.edgelist -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
        r = os.system(x)
        jsd = JS(datadir+'Football/football.edgelist',datadir+'Football/football.ecg','_embed',entropy=True)
        ## keep track of best and worst
        if jsd < best_jsd:
            os.system('cp _entropy _entropy_best')
            best_jsd = jsd
        if jsd > worst_jsd:
            os.system('cp _entropy _entropy_worst')
            worst_jsd = jsd

        ent = list(pd.read_csv('_entropy',header=None)[1])
        g.vs['ent'] = ent
        roc = roc_auc_score(g.vs['anomaly'], ent)
        L.append([dim,'n2v',str(p)+' '+str(q),jsd,roc])
D = pd.DataFrame(L,columns=['dim','algo','param','jsd','auc'])
D = D.sort_values(by='jsd',axis=0)
D.head()


In [ ]:
D.tail()

In [ ]:
## auc vs divergence (jsd)
plt.plot(D['jsd'],D['auc'],'o',color='black')
plt.xlabel('JS Divergence',fontsize=14)
plt.ylabel('AUC',fontsize=14);
#plt.savefig('anomaly_2.eps')

In [ ]:
## Entropy scores - some good embedding
g.vs['ent'] = list(pd.read_csv('_entropy_best',header=None)[1])
X = [v['ent'] for v in g.vs if v['anomaly']==0]
Y = [v['ent'] for v in g.vs if v['anomaly']==1]
plt.boxplot([X,Y],labels=['Regular','Anomalous'],sym='.',whis=(0,100), widths=.5)
plt.title("Low divergence embedding",fontsize=14)
plt.ylabel('Entropy',fontsize=14);
#plt.savefig('anomaly_3.eps')

In [ ]:
## Entropy scores - some not so good embedding
g.vs['ent'] = list(pd.read_csv('_entropy_worst',header=None)[1])
X = [v['ent'] for v in g.vs if v['anomaly']==0]
Y = [v['ent'] for v in g.vs if v['anomaly']==1]
plt.boxplot([X,Y],labels=['Regular','Anomalous'],sym='.',whis=(0,100), widths=.5)
plt.title("High divergence embedding",fontsize=14)
plt.ylabel('Entropy',fontsize=14);
#plt.savefig('anomaly_4.eps')

## AUC using average rank with several top embeddings

In [ ]:
from scipy.stats import rankdata
## try with top-k embeddings together - hopefully this consistently yields high AUC
k = 7
g.vs['rank'] = 0
for i in range(k):
    dim = D.iloc[i]['dim']
    p = float(D.iloc[i]['param'].split()[0])
    q = float(D.iloc[i]['param'].split()[1])
    x = 'node2vec -i:'+datadir+'Football/football.edgelist -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
    ## if you get unstable results, you can try with shorter random walks, for example:
    ## x = 'node2vec -l:15 -i:'+datadir+'Football/football.edgelist -o:_embed -d:'+str(dim)+' -p:'+str(p)+' -q:'+str(q)
    r = os.system(x)
    jsd = JS(datadir+'Football/football.edgelist',datadir+'Football/football.ecg','_embed',entropy=True)
    g.vs['ent'] = list(pd.read_csv('_entropy',header=None)[1])
    rk = rankdata(g.vs['ent'])
    for i in range(len(rk)):
        g.vs[i]['rank'] += rk[i]
print('AUC:',roc_auc_score(g.vs['anomaly'], g.vs['rank']))